In [93]:
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
base = pd.read_csv('/home/pa/Documents/github/doc_suomi/data/markov/markov.csv')

In [94]:
def splitter(data, group):
    data = list(data.groupby(group))
    data = [data[i][1] for i in range(len(data))]
    return data

In [95]:
base = base[base["energy"] != "start"]

In [96]:
le = preprocessing.LabelEncoder()
base['valence'] = le.fit_transform(base['valence'])
base['energy'] = le.fit_transform(base['energy'])
base['loudness'] = le.fit_transform(base['loudness'])
base['tempo'] = le.fit_transform(base['tempo'])

base["valence_next"] = le.fit_transform(base.groupby(["album_id"])["valence"].shift(-1))
base["energy_next"] = le.fit_transform(base.groupby(["album_id"])["energy"].shift(-1))
base["loudness_next"] = le.fit_transform(base.groupby(["album_id"])["loudness"].shift(-1))
base["tempo_next"] = le.fit_transform(base.groupby(["album_id"])["tempo"].shift(-1))

base = splitter(base, "album_id")
for i in base:
    i.reset_index(drop = True, inplace = True)



treino = base[0:int(len(base)*0.8)]
teste = base[len(treino):len(base)]

In [97]:
treino = pd.concat(treino)
teste = pd.concat(teste)

In [107]:
import numpy as np
from sklearn.dummy import DummyClassifier

feature = ["valence", "energy", "loudness", "tempo"]
out = ["valence_next", "energy_next", "loudness_next", "tempo_next"]
strategy = ["stratified", "most_frequent", "prior", "uniform"]

for k in range(len(feature)):
    print("-----------------------------------------------------")
    print("Feature: ", feature[k])
    for i in strategy:
        x_treino = treino[feature].values
        y_treino = treino[out[k]].values

        x_teste = teste[feature].values
        y_teste = teste[out[k]].values

        dummy_clf = DummyClassifier(strategy=i)

        dummy_clf.fit(x_treino, y_treino)

        pred = dummy_clf.predict(x_teste)
        print("Strategy:", i, ", Score:", round(accuracy_score(pred, y_teste), 2))


-----------------------------------------------------
Feature:  valence
Strategy: stratified , Score: 0.41
Strategy: most_frequent , Score: 0.46
Strategy: prior , Score: 0.46
Strategy: uniform , Score: 0.33
-----------------------------------------------------
Feature:  energy
Strategy: stratified , Score: 0.41
Strategy: most_frequent , Score: 0.46
Strategy: prior , Score: 0.46
Strategy: uniform , Score: 0.33
-----------------------------------------------------
Feature:  loudness
Strategy: stratified , Score: 0.42
Strategy: most_frequent , Score: 0.47
Strategy: prior , Score: 0.47
Strategy: uniform , Score: 0.32
-----------------------------------------------------
Feature:  tempo
Strategy: stratified , Score: 0.42
Strategy: most_frequent , Score: 0.46
Strategy: prior , Score: 0.46
Strategy: uniform , Score: 0.34
